# Control of a "spacecraft" platform with a reaction wheel

Import modules and configure the notebook.

In [ ]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
import sympy as sym
import matplotlib.pyplot as plt

# This is a custom interface to the pybullet simulator
import ae353_platform

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are several optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `damping` (a non-negative floating-point number, default `0.`) is the coefficient of viscous friction at each joint;
* `roll` (an angle in radians, default `np.pi / 2`) is the roll angle of the ground.

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug.

Given a choice of ground roll angle $\phi$, the equations of motion are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)
\end{aligned}
$$

where

* $q_1$ is the platform angle
* $\dot{q}_1$ is the platform velocity
* $v_2$ is the wheel velocity
* $\tau$ is the torque applied *to* the platform *by* the wheel

and

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

and parameter values are defined as follows:

In [ ]:
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau')

In [ ]:
f = sym.Matrix([
    v1,
    (1 / J1) * (tau - mw * g * l * sym.sin(q1)),
    (1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))
])

In [ ]:
f

In [ ]:
q1e = 0.
v1e = 0.
v2e = 0.
taue = 0.

f_num = sym.lambdify([q1, v1, v2, tau], f)

In [ ]:
f_num(q1e, v1e, v2e, taue)

In [ ]:
simulator = ae353_platform.Simulator(
    display=True,
    damping=0.,
    roll=(np.pi / 2),
)

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

# composite parameters
J1 = Jp + mw * l**2
J2 = Jw

Define a controller that maps sensor measurements to actuator commands. By default, it chooses to apply zero torque to the wheel. Modify the `run` function to make the controller behave differently.

In [ ]:
class Controller:
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            wheel_angle,
            wheel_velocity,
        ):
        
        # FIXME: replace the following line
        wheel_torque = 2.
        
        return wheel_torque

Create an instance of the controller.

In [ ]:
controller = Controller()

Reset the simulator with given initial conditions. A low-level controller (separate from yours) will try to ensure that the rotor velocity remains constant.

In [ ]:
simulator.reset(
    platform_angle=np.pi/3,
    platform_velocity=0.,
    wheel_angle=0.,
    wheel_velocity=0.,
)

Reset the controller.

In [ ]:
controller.reset()

Choose a camera view (could be `simulator.camera_topview()` or `simulator.camera_sideview()`).

In [ ]:
simulator.camera_topview()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=5.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot the results.

In [ ]:
# Create a figure with four subplots, all of which share the same x-axis
fig, (
        ax_platform_angle,
        ax_platform_velocity,
        ax_wheel,
        ax_torque
    ) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

# Platform angle
ax_platform_angle.plot(
    data['t'], data['platform_angle'],
    label='platform angle (rad)', linewidth=4,
)
ax_platform_angle.grid()
ax_platform_angle.legend(fontsize=16)
ax_platform_angle.tick_params(labelsize=14)

# Platform velocity
ax_platform_velocity.plot(
    data['t'], data['platform_velocity'],
    label='platform velocity (rad / s)', linewidth=4,
)
ax_platform_velocity.grid()
ax_platform_velocity.legend(fontsize=16)
ax_platform_velocity.tick_params(labelsize=14)

# Wheel velocity
ax_wheel.plot(
    data['t'], data['wheel_velocity'],
    label='wheel velocity (rad / s)', linewidth=4,
)
ax_wheel.grid()
ax_wheel.legend(fontsize=16)
ax_wheel.tick_params(labelsize=14)

# Wheel torque
ax_torque.plot(
    data['t'], data['wheel_torque_command'],
    label='wheel torque command (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['wheel_torque'],
    '--', label='wheel torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], np.ones_like(data['t']) * simulator.tau_max,
    ':', label='max torque', linewidth=4, color='C2', zorder=0,
)
ax_torque.plot(
    data['t'], -np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C2', zorder=0,
)
ax_torque.grid()
ax_torque.legend(fontsize=16)
ax_torque.tick_params(labelsize=14)

# Set x-axis properties (only need to do this on the last
# subplot since all subplots share the same x-axis)
ax_torque.set_xlabel('time (s)', fontsize=20)
ax_torque.set_xlim([data['t'][0], data['t'][-1]])
ax_torque.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

# Make the arrangement of subplots look nice
fig.tight_layout()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)